In [1]:
import pandas as pd
import numpy as np
import pickle 

# function to read data from file
def read_dataset(file_name,language):
    path="../data/"+language+"/"+file_name+".conllu"
    data = pd.read_csv (path, sep = '\t',quoting=3, names=["POSITION","WORD","TAG"])
    return data

train_data=read_dataset("train","en")
print(train_data)

         POSITION      WORD TAG
0               0      This   O
1               1  division   O
2               2      also   O
3               3  contains   O
4               4       the   O
...           ...       ...  ..
2193674        27      born   O
2193675        28         1   O
2193676        29       May   O
2193677        30      1964   O
2193678        31         .   O

[2193679 rows x 3 columns]


# LEARNING:
### Counting
- word_tag_counts = number of times a word is associated with each tag
- tag_tag_counts =  number of times a tag is followed by another tag    (also considering an extra tag "START")

In [9]:
def counts_occurence(train_data):
  tag_list=train_data['TAG'].unique()
  word_list=train_data['WORD'].unique()
  empty_tag_count_dict= {tag:0 for tag in tag_list}

  tag_counts = empty_tag_count_dict.copy()  # {tag1:0, tag2:0, ...}
  word_tag_counts = {word:empty_tag_count_dict.copy() for word in word_list} # {word1:{tag1:0, tag2:0, ...}, word2:{tag1:0, tag2:0, ...}, ...}
  tag_tag_counts = {tag:empty_tag_count_dict.copy() for tag in tag_list} # {tag1:{tag1:0, tag2:0, ...}, tag2:{tag1:0, tag2:0, ...}, ...}
  #tag_tag_counts['START'] = empty_tag_count_dict.copy()
  #tag_counts['START'] = 0
 

  for index, row in train_data.iterrows():
      word = row['WORD']
      tag = row['TAG']
      pos= row['POSITION']
      
      if pos!=0:
        tag_tag_counts[prev_tag][tag] +=1

      
      tag_counts[tag] += 1
      word_tag_counts[word][tag] +=1
      prev_tag = tag

  
  return tag_counts, word_tag_counts, tag_tag_counts

tag_counts, word_tag_counts, tag_tag_counts= counts_occurence(train_data)



{'O': {'O': 1692935, 'B-LOC': 52617, 'I-LOC': 0, 'B-ORG': 23152, 'B-MISC': 34187, 'I-MISC': 3831, 'B-PER': 36663, 'I-PER': 0, 'I-ORG': 0}, 'B-LOC': {'O': 40581, 'B-LOC': 134, 'I-LOC': 14174, 'B-ORG': 0, 'B-MISC': 0, 'I-MISC': 0, 'B-PER': 10, 'I-PER': 3, 'I-ORG': 0}, 'I-LOC': {'O': 14163, 'B-LOC': 4, 'I-LOC': 4592, 'B-ORG': 1, 'B-MISC': 1, 'I-MISC': 0, 'B-PER': 6, 'I-PER': 0, 'I-ORG': 0}, 'B-ORG': {'O': 12196, 'B-LOC': 0, 'I-LOC': 0, 'B-ORG': 52, 'B-MISC': 0, 'I-MISC': 0, 'B-PER': 11, 'I-PER': 0, 'I-ORG': 11966}, 'B-MISC': {'O': 23679, 'B-LOC': 0, 'I-LOC': 0, 'B-ORG': 0, 'B-MISC': 821, 'I-MISC': 11644, 'B-PER': 18, 'I-PER': 1, 'I-ORG': 0}, 'I-MISC': {'O': 15455, 'B-LOC': 0, 'I-LOC': 1, 'B-ORG': 0, 'B-MISC': 17, 'I-MISC': 17915, 'B-PER': 0, 'I-PER': 2, 'I-ORG': 1}, 'B-PER': {'O': 14419, 'B-LOC': 0, 'I-LOC': 0, 'B-ORG': 0, 'B-MISC': 0, 'I-MISC': 0, 'B-PER': 14, 'I-PER': 25831, 'I-ORG': 0}, 'I-PER': {'O': 25833, 'B-LOC': 0, 'I-LOC': 0, 'B-ORG': 0, 'B-MISC': 0, 'I-MISC': 1, 'B-PER': 3, 'I-P

### Probabilities
- emission_prob = probability, given a tag, that it will be associated with a given word
- transition_prob = probability of a tag occurring given the previous tag

In [3]:
def calculate_probs(tag_counts, word_tag_counts, tag_tag_counts):
    tag_list=train_data['TAG'].unique()
    word_list=train_data['WORD'].unique()
    empty_tag_count_dict= {tag:0 for tag in tag_list}

    emission_prob = {word:empty_tag_count_dict.copy() for word in word_list} 
    transition_prob= {tag:empty_tag_count_dict.copy() for tag in tag_list} 
    #transition_prob['START'] = empty_tag_count_dict.copy()

    for word, tag_dict in word_tag_counts.items():
        for tag in tag_dict:
            if word_tag_counts[word][tag]!=0:
                emission_prob[word][tag] = np.log(word_tag_counts[word][tag]/tag_counts[tag])                    
            else:
                emission_prob[word][tag] = 0
        
    for prev, tag_dict in tag_tag_counts.items():
        for next in tag_dict:
            if tag_tag_counts[prev][next]!=0:
                transition_prob[prev][next] = np.log(tag_tag_counts[prev][next]/tag_counts[prev])
            else:
                transition_prob[prev][next] = 0
    return emission_prob, transition_prob
    
emission_prob, transition_prob= calculate_probs(tag_counts, word_tag_counts, tag_tag_counts)

In [4]:
###CODICE TEST DA BUTTARE,PER VEDERE LA SOMMA DELLE PROBABILITÀ: non è esattamente uno a causa di approssimazioni, spero non crei problemi

emission_dataframe=pd.DataFrame.from_dict(emission_prob)
transition_dataframe=pd.DataFrame.from_dict(transition_prob)

print(emission_dataframe)
#tag_sums1 = {}
#tag_sums2 = {}
#for word in emission_prob:
#    for tag, value in emission_prob[word].items():
#        tag_sums1[tag] = tag_sums1.get(tag, 0) + value
#    print("emission",tag_sums1)

#for tag in transition_prob:
#     for next_tag, value in transition_prob[tag].items():
#         tag_sums2[tag] = tag_sums2.get(tag, 0) + value
#     print("transition",tag_sums2)



             This  division      also  contains        the    Ventana  \
O       -6.732052 -9.069017 -5.812474 -9.016603  -2.777174   0.000000   
B-LOC    0.000000  0.000000  0.000000  0.000000  -8.428398 -10.220158   
I-LOC    0.000000  0.000000  0.000000  0.000000  -5.796804   0.000000   
B-ORG    0.000000  0.000000  0.000000  0.000000  -5.875633   0.000000   
B-MISC  -7.930842  0.000000  0.000000  0.000000  -8.097897   0.000000   
I-MISC  -8.336600  0.000000  0.000000  0.000000  -3.529510   0.000000   
B-PER  -10.603213  0.000000  0.000000  0.000000 -10.603213   0.000000   
I-PER  -10.290992  0.000000  0.000000  0.000000  -5.423458   0.000000   
I-ORG    0.000000  0.000000  0.000000  0.000000  -4.817149   0.000000   

        Wilderness         ,      home        to  ...       Rapp  shanklish  \
O         0.000000 -2.836472 -7.562451 -3.932190  ...   0.000000 -14.476189   
B-LOC     0.000000  0.000000  0.000000  0.000000  ...   0.000000   0.000000   
I-LOC    -7.537270 -9.839855  0.

In [5]:
#serialize data into a file 
def save_data(data,file_name,language):
    path="../data/"+language+"/"+file_name
    try: 
        file= open(path, 'wb') 
        pickle.dump(data, file) 
        file.close() 
    except: 
        print("Error in writing data")

# save_data(transition_prob,"transition_prob","it")

In [6]:
#Learn emission and transition probabilities for each language
for language in ["en","it","es"]:
    read_dataset("train",language)
    tag_counts, word_tag_counts, tag_tag_counts= counts_occurence(train_data)
    emission_prob, transition_prob= calculate_probs(tag_counts, word_tag_counts, tag_tag_counts)
    save_data(transition_prob,"transition_prob",language)
    save_data(emission_prob,"emission_prob",language)